In [1]:
# https://developer.spotify.com/console/get-audio-features-several-tracks/?ids=4JpKVNYnVcJ8tuMKjAj50A,2NRANZE9UCmPAS5XVbXL40,24JygzOLM0EmRQeGtFcIcG

#https://www.kaggle.com/aeryan/spotify-music-analysis

#https://www.kaggle.com/cihanoklap/top-songs-on-spotify-what-makes-them-popular

In [2]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo
import json

# pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
path = "Charts CSV/regional-global-daily-latest.csv"
spotify_data_global = pd.read_csv(path)
spotify_data_global.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e
2,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
3,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
4,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE


In [4]:
spotify_data_global.to_csv("clean_global.csv", header=False, index=False)


In [5]:
path_clean = "clean_global.csv"
spotify_data_global_clean = pd.read_csv(path_clean)
spotify_data_global_clean.head()

,Position,Track Name,Artist,Streams,URL
0,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e
1,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
2,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
3,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE
4,5,Kiss Me More (feat. SZA),Doja Cat,4217120,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz


In [6]:
spotify_data_global_clean["ID"] = spotify_data_global_clean["URL"].str[31:]
spotify_data_global_clean.head()

,Position,Track Name,Artist,Streams,URL,ID
0,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e,3Wrjm47oTz2sjIgck11l5e
1,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20,5HCyWlXZPP0y6Gqq8TgA20
2,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG,4ZtFanR9U6ndgddUvNcjcG
3,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE,6PQ88X9TkUIAUIZJHW2upE
4,5,Kiss Me More (feat. SZA),Doja Cat,4217120,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz,3DarAbFujv6eYNliUTyqtz


In [7]:
len("https://open.spotify.com/track/")

31

In [8]:
test = spotify_data_global_clean["URL"].str
print(test[31:])

0      3Wrjm47oTz2sjIgck11l5e
1      5HCyWlXZPP0y6Gqq8TgA20
2      4ZtFanR9U6ndgddUvNcjcG
3      6PQ88X9TkUIAUIZJHW2upE
4      3DarAbFujv6eYNliUTyqtz
                ...          
195    5EVzgDPaM0LCM3M5exl3z8
196    7fBv7CLKzipRk6EC6TWHOB
197    5zqObw7wjBgL9TDiAymxPn
198    3rq5w4bQGigXOfdN30ATJt
199    4JeOesjcWWlprTAvucDSJX
Name: URL, Length: 200, dtype: object


In [14]:
# token = "BQAEIQPWvNjgZD41CmvBSrkEvOTxVYxWU1l9GaAFb79KuLLanpGlLTccAsdvVz5yK-xzOFVzHnnDYARd-eZbZGKsuQU0Yt8MofmTOiPCW3CRv6PZYmcxHro-BQ25W6gUcbyd6YTxWEccl9-1UP8SNAArj7IjeQ"
# token = "BQC7SbbuUwwebllY8WlsJYjDKwpMeTF8twCsti47Or50rxWXZ0JOYs7IXIrDOb6LnhE5YPUu_nUkkjTma-zh6xPjf_j7AtCIEk4qOdQu8i_wvvjie5Bp1xANNhhhjusybp6spB2ZFhbJ-0dTfVE"
# token = "BQB1hjVmwHas7EQTBD5SgV0fYBJFhkQdLjo6QV9qMxtwM-KWxVOcVt1f1934NQ2wPT4xZ27TQVpUsQV5JG-GR9-GJoE1iAvqR32bHL4ttyRNRgIChRd4sCfUoFeSMhodlYUUv5Tys9sVJk-fdWNXNg"
#token="BQAqFieWd9Vs7_PadOTnr5aa5Z718R_mmO6WACUnALMgjtzORXuE1eV6wxBonSkIlbKt924WCoIdF2Ia9QSgFlKzw2VONVEbgzTYTMUobMQTP4CSybOuLgCyAc8dasFXh8ycBwfmtLKvrHb5D9v7c3t_DxahFixpPZd-TsBPoCbRgw"
token = "BQB7MQS25CAJBY0_A7cLn1k-hjxKiJ9C5efP0m2gS-4FM1O3ySSJjQfZuUktBUKcqv14KmVnLYTmK8tpYLU-KmK9FYUsRl26OzMKrKufqtgCGOaMStQhGA8ykkmM_fBqNdt1LC4kPqj5GoAe7i9GNwcIM57p_guU93xZ1F44eEPvRzQCwgWG_vfWrR0yPHvhPVHxlVWlnvMGuB_al3cpiNWABHRCAI4SNsy15u_CYPXSq6FR047cFkWO_Ijwd5yjeoyz8k4ho5gQL3Oe3geX"
headers_dict = {"Authorization": f'Bearer {token}'}

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.SpotifyTopChartsdb
collection = db.TopGlobal
collection.drop()

rows=len(spotify_data_global_clean)
global_dict={}
# spotify_artist_dict=[]

for row in range(0,rows):
    # Hacer el llamado a la API de Spotify usando spotify_data_wNA.iloc[row,4]
    
    
    artist_id = spotify_data_global_clean.iloc[row,5]
    artist_name = spotify_data_global_clean.iloc[row,2]
    song_name = spotify_data_global_clean.iloc[row,1]

    
    response = requests.get(f'https://api.spotify.com/v1/audio-features?ids={artist_id}', headers=headers_dict).json()
    print(response)
    # artist_id = response["audio_features"]
    print(response["audio_features"])

    danceability = response["audio_features"][0]["danceability"]
    energy = response["audio_features"][0]["energy"]
    key = response["audio_features"][0]["liveness"]
    loudness = response["audio_features"][0]["key"]
    acousticness = response["audio_features"][0]["acousticness"]
    instrumentalness = response["audio_features"][0]["instrumentalness"]
    liveness = response["audio_features"][0]["liveness"]
    valence = response["audio_features"][0]["valence"]
    tempo = response["audio_features"][0]["tempo"]
    track_href = response["audio_features"][0]["track_href"]
        
    print(artist_name)    
    global_dict={'Artist Name': artist_name,
                 'Song Name': song_name,
                'Danceability': danceability,
                 'Energy': energy,
                 'Key': key,
                 'Loudness': loudness,
                 'Acousticness': acousticness,
                 'Instrumentalness': instrumentalness,
                 'Liveness': liveness,
                 'Valence': valence,
                 'Tempo': tempo,
                 'Track Reference': track_href
                 }
    collection.insert_one(global_dict)
#     spotify_artist_dict.append(artist_dict)

{'audio_features': [{'danceability': 0.714, 'energy': 0.8, 'key': 11, 'loudness': -4.808, 'mode': 0, 'speechiness': 0.0504, 'acousticness': 0.127, 'instrumentalness': 0, 'liveness': 0.359, 'valence': 0.589, 'tempo': 134.002, 'type': 'audio_features', 'id': '3Wrjm47oTz2sjIgck11l5e', 'uri': 'spotify:track:3Wrjm47oTz2sjIgck11l5e', 'track_href': 'https://api.spotify.com/v1/tracks/3Wrjm47oTz2sjIgck11l5e', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3Wrjm47oTz2sjIgck11l5e', 'duration_ms': 211560, 'time_signature': 4}]}
[{'danceability': 0.714, 'energy': 0.8, 'key': 11, 'loudness': -4.808, 'mode': 0, 'speechiness': 0.0504, 'acousticness': 0.127, 'instrumentalness': 0, 'liveness': 0.359, 'valence': 0.589, 'tempo': 134.002, 'type': 'audio_features', 'id': '3Wrjm47oTz2sjIgck11l5e', 'uri': 'spotify:track:3Wrjm47oTz2sjIgck11l5e', 'track_href': 'https://api.spotify.com/v1/tracks/3Wrjm47oTz2sjIgck11l5e', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3Wrjm47oTz2sjIgck11l5e

{'audio_features': [{'danceability': 0.644, 'energy': 0.648, 'key': 1, 'loudness': -4.601, 'mode': 1, 'speechiness': 0.118, 'acousticness': 0.276, 'instrumentalness': 0, 'liveness': 0.135, 'valence': 0.44, 'tempo': 179.951, 'type': 'audio_features', 'id': '2JPLbjOn0wPCngEot2STUS', 'uri': 'spotify:track:2JPLbjOn0wPCngEot2STUS', 'track_href': 'https://api.spotify.com/v1/tracks/2JPLbjOn0wPCngEot2STUS', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2JPLbjOn0wPCngEot2STUS', 'duration_ms': 206710, 'time_signature': 4}]}
[{'danceability': 0.644, 'energy': 0.648, 'key': 1, 'loudness': -4.601, 'mode': 1, 'speechiness': 0.118, 'acousticness': 0.276, 'instrumentalness': 0, 'liveness': 0.135, 'valence': 0.44, 'tempo': 179.951, 'type': 'audio_features', 'id': '2JPLbjOn0wPCngEot2STUS', 'uri': 'spotify:track:2JPLbjOn0wPCngEot2STUS', 'track_href': 'https://api.spotify.com/v1/tracks/2JPLbjOn0wPCngEot2STUS', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2JPLbjOn0wPCngEot2STUS',

{'audio_features': [{'danceability': 0.659, 'energy': 0.688, 'key': 6, 'loudness': -4.649, 'mode': 1, 'speechiness': 0.0673, 'acousticness': 0.207, 'instrumentalness': 0, 'liveness': 0.113, 'valence': 0.632, 'tempo': 153.979, 'type': 'audio_features', 'id': '0G2zPzWqVjR68iNPmx2TBe', 'uri': 'spotify:track:0G2zPzWqVjR68iNPmx2TBe', 'track_href': 'https://api.spotify.com/v1/tracks/0G2zPzWqVjR68iNPmx2TBe', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0G2zPzWqVjR68iNPmx2TBe', 'duration_ms': 273474, 'time_signature': 4}]}
[{'danceability': 0.659, 'energy': 0.688, 'key': 6, 'loudness': -4.649, 'mode': 1, 'speechiness': 0.0673, 'acousticness': 0.207, 'instrumentalness': 0, 'liveness': 0.113, 'valence': 0.632, 'tempo': 153.979, 'type': 'audio_features', 'id': '0G2zPzWqVjR68iNPmx2TBe', 'uri': 'spotify:track:0G2zPzWqVjR68iNPmx2TBe', 'track_href': 'https://api.spotify.com/v1/tracks/0G2zPzWqVjR68iNPmx2TBe', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0G2zPzWqVjR68iNPmx2T

{'audio_features': [{'danceability': 0.824, 'energy': 0.862, 'key': 2, 'loudness': -3.424, 'mode': 1, 'speechiness': 0.126, 'acousticness': 0.0076, 'instrumentalness': 0.000132, 'liveness': 0.303, 'valence': 0.801, 'tempo': 122.98, 'type': 'audio_features', 'id': '4cG7HUWYHBV6R6tHn1gxrl', 'uri': 'spotify:track:4cG7HUWYHBV6R6tHn1gxrl', 'track_href': 'https://api.spotify.com/v1/tracks/4cG7HUWYHBV6R6tHn1gxrl', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4cG7HUWYHBV6R6tHn1gxrl', 'duration_ms': 169153, 'time_signature': 4}]}
[{'danceability': 0.824, 'energy': 0.862, 'key': 2, 'loudness': -3.424, 'mode': 1, 'speechiness': 0.126, 'acousticness': 0.0076, 'instrumentalness': 0.000132, 'liveness': 0.303, 'valence': 0.801, 'tempo': 122.98, 'type': 'audio_features', 'id': '4cG7HUWYHBV6R6tHn1gxrl', 'uri': 'spotify:track:4cG7HUWYHBV6R6tHn1gxrl', 'track_href': 'https://api.spotify.com/v1/tracks/4cG7HUWYHBV6R6tHn1gxrl', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4cG7HUWY

{'audio_features': [{'danceability': 0.789, 'energy': 0.536, 'key': 6, 'loudness': -6.862, 'mode': 1, 'speechiness': 0.242, 'acousticness': 0.41, 'instrumentalness': 0, 'liveness': 0.129, 'valence': 0.437, 'tempo': 81.039, 'type': 'audio_features', 'id': '18vXApRmJSgQ6wG2ll9AOg', 'uri': 'spotify:track:18vXApRmJSgQ6wG2ll9AOg', 'track_href': 'https://api.spotify.com/v1/tracks/18vXApRmJSgQ6wG2ll9AOg', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/18vXApRmJSgQ6wG2ll9AOg', 'duration_ms': 165926, 'time_signature': 4}]}
[{'danceability': 0.789, 'energy': 0.536, 'key': 6, 'loudness': -6.862, 'mode': 1, 'speechiness': 0.242, 'acousticness': 0.41, 'instrumentalness': 0, 'liveness': 0.129, 'valence': 0.437, 'tempo': 81.039, 'type': 'audio_features', 'id': '18vXApRmJSgQ6wG2ll9AOg', 'uri': 'spotify:track:18vXApRmJSgQ6wG2ll9AOg', 'track_href': 'https://api.spotify.com/v1/tracks/18vXApRmJSgQ6wG2ll9AOg', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/18vXApRmJSgQ6wG2ll9AOg', '

...

{'audio_features': [{'danceability': 0.414, 'energy': 0.404, 'key': 0, 'loudness': -9.928, 'mode': 0, 'speechiness': 0.0499, 'acousticness': 0.271, 'instrumentalness': 0, 'liveness': 0.3, 'valence': 0.224, 'tempo': 71.105, 'type': 'audio_features', 'id': '4u7EnebtmKWzUH433cf5Qv', 'uri': 'spotify:track:4u7EnebtmKWzUH433cf5Qv', 'track_href': 'https://api.spotify.com/v1/tracks/4u7EnebtmKWzUH433cf5Qv', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4u7EnebtmKWzUH433cf5Qv', 'duration_ms': 354320, 'time_signature': 4}]}
[{'danceability': 0.414, 'energy': 0.404, 'key': 0, 'loudness': -9.928, 'mode': 0, 'speechiness': 0.0499, 'acousticness': 0.271, 'instrumentalness': 0, 'liveness': 0.3, 'valence': 0.224, 'tempo': 71.105, 'type': 'audio_features', 'id': '4u7EnebtmKWzUH433cf5Qv', 'uri': 'spotify:track:4u7EnebtmKWzUH433cf5Qv', 'track_href': 'https://api.spotify.com/v1/tracks/4u7EnebtmKWzUH433cf5Qv', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4u7EnebtmKWzUH433cf5Qv', '

{'audio_features': [{'danceability': 0.548, 'energy': 0.805, 'key': 7, 'loudness': -5.732, 'mode': 1, 'speechiness': 0.351, 'acousticness': 0.212, 'instrumentalness': 0.000389, 'liveness': 0.408, 'valence': 0.648, 'tempo': 142.094, 'type': 'audio_features', 'id': '79s5XnCN4TJKTVMSmOx8Ep', 'uri': 'spotify:track:79s5XnCN4TJKTVMSmOx8Ep', 'track_href': 'https://api.spotify.com/v1/tracks/79s5XnCN4TJKTVMSmOx8Ep', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/79s5XnCN4TJKTVMSmOx8Ep', 'duration_ms': 216387, 'time_signature': 4}]}
[{'danceability': 0.548, 'energy': 0.805, 'key': 7, 'loudness': -5.732, 'mode': 1, 'speechiness': 0.351, 'acousticness': 0.212, 'instrumentalness': 0.000389, 'liveness': 0.408, 'valence': 0.648, 'tempo': 142.094, 'type': 'audio_features', 'id': '79s5XnCN4TJKTVMSmOx8Ep', 'uri': 'spotify:track:79s5XnCN4TJKTVMSmOx8Ep', 'track_href': 'https://api.spotify.com/v1/tracks/79s5XnCN4TJKTVMSmOx8Ep', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/79s5XnCN

{'audio_features': [{'danceability': 0.628, 'energy': 0.581, 'key': 2, 'loudness': -8.377, 'mode': 1, 'speechiness': 0.188, 'acousticness': 0.46, 'instrumentalness': 0, 'liveness': 0.152, 'valence': 0.259, 'tempo': 145.396, 'type': 'audio_features', 'id': '2wzionesqIt3x5Sglb4n91', 'uri': 'spotify:track:2wzionesqIt3x5Sglb4n91', 'track_href': 'https://api.spotify.com/v1/tracks/2wzionesqIt3x5Sglb4n91', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2wzionesqIt3x5Sglb4n91', 'duration_ms': 120828, 'time_signature': 4}]}
[{'danceability': 0.628, 'energy': 0.581, 'key': 2, 'loudness': -8.377, 'mode': 1, 'speechiness': 0.188, 'acousticness': 0.46, 'instrumentalness': 0, 'liveness': 0.152, 'valence': 0.259, 'tempo': 145.396, 'type': 'audio_features', 'id': '2wzionesqIt3x5Sglb4n91', 'uri': 'spotify:track:2wzionesqIt3x5Sglb4n91', 'track_href': 'https://api.spotify.com/v1/tracks/2wzionesqIt3x5Sglb4n91', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2wzionesqIt3x5Sglb4n91',

{'audio_features': [{'danceability': 0.83, 'energy': 0.517, 'key': 6, 'loudness': -6.416, 'mode': 0, 'speechiness': 0.333, 'acousticness': 0.399, 'instrumentalness': 8.95e-05, 'liveness': 0.0916, 'valence': 0.616, 'tempo': 171.959, 'type': 'audio_features', 'id': '2dNhNlpZAtV4lwAFW45LIv', 'uri': 'spotify:track:2dNhNlpZAtV4lwAFW45LIv', 'track_href': 'https://api.spotify.com/v1/tracks/2dNhNlpZAtV4lwAFW45LIv', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2dNhNlpZAtV4lwAFW45LIv', 'duration_ms': 182496, 'time_signature': 4}]}
[{'danceability': 0.83, 'energy': 0.517, 'key': 6, 'loudness': -6.416, 'mode': 0, 'speechiness': 0.333, 'acousticness': 0.399, 'instrumentalness': 8.95e-05, 'liveness': 0.0916, 'valence': 0.616, 'tempo': 171.959, 'type': 'audio_features', 'id': '2dNhNlpZAtV4lwAFW45LIv', 'uri': 'spotify:track:2dNhNlpZAtV4lwAFW45LIv', 'track_href': 'https://api.spotify.com/v1/tracks/2dNhNlpZAtV4lwAFW45LIv', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2dNhNlpZ

{'audio_features': [{'danceability': 0.933, 'energy': 0.528, 'key': 5, 'loudness': -6.472, 'mode': 0, 'speechiness': 0.161, 'acousticness': 0.112, 'instrumentalness': 0.312, 'liveness': 0.163, 'valence': 0.754, 'tempo': 109.967, 'type': 'audio_features', 'id': '5vdp5UmvTsnMEMESIF2Ym7', 'uri': 'spotify:track:5vdp5UmvTsnMEMESIF2Ym7', 'track_href': 'https://api.spotify.com/v1/tracks/5vdp5UmvTsnMEMESIF2Ym7', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5vdp5UmvTsnMEMESIF2Ym7', 'duration_ms': 214653, 'time_signature': 4}]}
[{'danceability': 0.933, 'energy': 0.528, 'key': 5, 'loudness': -6.472, 'mode': 0, 'speechiness': 0.161, 'acousticness': 0.112, 'instrumentalness': 0.312, 'liveness': 0.163, 'valence': 0.754, 'tempo': 109.967, 'type': 'audio_features', 'id': '5vdp5UmvTsnMEMESIF2Ym7', 'uri': 'spotify:track:5vdp5UmvTsnMEMESIF2Ym7', 'track_href': 'https://api.spotify.com/v1/tracks/5vdp5UmvTsnMEMESIF2Ym7', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5vdp5UmvTsnMEM